<a href="https://colab.research.google.com/github/HackerJacky/TimeSeriesColab/blob/main/TS_HW7_2model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import yfinance as yf
from scipy import stats
from google.colab import drive
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv('/content/monthly_ret_sp500_TW.csv')
data.index=pd.to_datetime(data['Date'])
data.head()

,Date,ret_sp500,ret_TW_index,Australia,Brazil,France,Singapore,US
Date,,,,,,,,
2018-01-31,2018/1/31,0.046388,0.036040,48.901424,165.005041,221.137569,146.146055,160.967205
2018-02-28,2018/2/28,-0.039726,-0.026309,67.914075,68.463581,192.030587,122.308889,114.755558
2018-03-31,2018/3/31,-0.027252,0.008356,84.093319,101.205300,195.572139,168.097881,187.095890
2018-04-30,2018/4/30,0.002715,-0.023034,48.264544,152.250952,187.361105,156.113780,139.035386
2018-05-31,2018/5/31,0.021378,0.020163,75.622559,179.808920,303.053908,176.825324,162.276697


| 證據來源                         | 闡述內容                                              |
| ---------------------------- | ------------------------------------------------- |
| **X 含 year/quarter dummies** | 顯示模型目的並非即期交易反應，而是針對**結構性預測能力分析**                  |
| **ΔEPU 是滯後項**                | 表示要觀察不確定性「持續時間與延遲反應」對未來的影響                        |
| **Newey-West 標準誤**           | 預設處理中期影響的 heteroskedasticity & autocorrelation 問題 |


所以這種模型設計偏向「年度或季度趨勢預測」,用 window = 12 最能對應此目的與架構。



###補充視窗大小

| 視窗長度    | 理論涵義               | 實務效果                 | 適用目的                | 顯著性表現（你模型中）                         |
| ------- | ------------------ | -------------------- | ------------------- | ----------------------------------- |
| **3**   | 捕捉短期波動、反映市場情緒與即時衝擊 | 高敏感性、但波動大、易受雜訊影響     | 短期事件反應、突發政策、恐慌情緒    | 多數變數不顯著，解釋力低                        |
| **6**   | 平衡即期衝擊與結構趨勢        | 較穩定，仍能部分抓到事件影響       | 中期調整觀察、波段級事件影響      | 開始出現 `sigma_lag3` 顯著                |
| **12**  | 年度週期考量，對商業與財報週期有意義 | 抑制短期雜訊、捕捉年內聯動        | 評估跨年政策或財報週期對聯動關係的影響 | `lag3` 顯著性更高                        |
| **18**  | 長期共移、結構性金融連動（近一年半） | 變數平滑，統計穩定性高，受短期雜訊干擾少 | 金融整合、風險溢出、長期聯動關係分析  | `sigma_lag3`、`corr_TW_US_lag3` 多國顯著 |
| **24+** | 超長期結構共變異，適用於資金趨勢   | 平滑過頭，靈敏度下降、易忽略局部事件   | 大型資產配置趨勢研究、主權基金視角   | 可能過度平滑，顯著性反而降低                      |


In [ ]:
data['sigma_sp500'] = data['ret_sp500'].rolling(window=12).std()

In [ ]:
for country in ['Australia', 'Brazil', 'France', 'Singapore', 'US']:
    data[f'{country}_pct'] = data[country].pct_change()
    for lag in range(1, 4):
        data[f'{country}_pct_lag{lag}'] = data[country].pct_change().shift(lag)


for j in range(1, 4):
    data[f'sigma_sp500_lag{j}'] = data['sigma_sp500'].shift(j)

data.head()

,Date,ret_sp500,ret_TW_index,Australia,Brazil,France,Singapore,US,Australia_pct,Australia_pct_lag1,...,sigma_sp500_lag2,sigma_sp500_lag3,sigma_sp500_lead1,dummy_event_march,dummy_post_q2,corr_TW_US,corr_TW_US_lag1,corr_TW_US_lag2,corr_TW_US_lag3,corr_TW_US_lead1
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,2018/1/31,0.046388,0.036040,48.901424,165.005041,221.137569,146.146055,160.967205,NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2018-02-28,2018/2/28,-0.039726,-0.026309,67.914075,68.463581,192.030587,122.308889,114.755558,0.388795,NaN,...,NaN,NaN,0.046537,0,0,NaN,NaN,NaN,NaN,NaN
2018-03-31,2018/3/31,-0.027252,0.008356,84.093319,101.205300,195.572139,168.097881,187.095890,0.238231,0.388795,...,NaN,NaN,0.021813,0,0,NaN,NaN,NaN,NaN,NaN
2018-04-30,2018/4/30,0.002715,-0.023034,48.264544,152.250952,187.361105,156.113780,139.035386,-0.426060,0.238231,...,NaN,NaN,0.024533,0,0,NaN,NaN,NaN,NaN,NaN
2018-05-31,2018/5/31,0.021378,0.020163,75.622559,179.808920,303.053908,176.825324,162.276697,0.566835,-0.426060,...,NaN,NaN,0.010219,0,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
data['sigma_sp500_lead1'] = data['sigma_sp500'].shift(-1)

###判斷顯著


In [ ]:
y=data['ret_sp500']
X=data[['Australia_pct_lag1','Australia_pct_lag2','Australia_pct_lag3','sigma_sp500_lag1','sigma_sp500_lag2','sigma_sp500_lag3']]
x=sm.add_constant(X)
model=sm.OLS(y,x,missing='drop').fit()
print(model.summary())
#澳洲整體數據不顯著

                            OLS Regression Results                            
Dep. Variable:              ret_sp500   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                 -0.033
Method:                 Least Squares   F-statistic:                    0.6150
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.717
Time:                        16:04:24   Log-Likelihood:                 117.29
No. Observations:                  73   AIC:                            -220.6
Df Residuals:                      66   BIC:                            -204.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.0330      0

In [ ]:
y=data['ret_sp500']
X=data[['Brazil_pct_lag1','Brazil_pct_lag2','Brazil_pct_lag3','sigma_sp500_lag1','sigma_sp500_lag2','sigma_sp500_lag3']]
x=sm.add_constant(X)
model=sm.OLS(y,x,missing='drop').fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              ret_sp500   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     1.371
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.239
Time:                        16:04:27   Log-Likelihood:                 119.59
No. Observations:                  73   AIC:                            -225.2
Df Residuals:                      66   BIC:                            -209.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0362      0.026  

In [ ]:
y=data['ret_sp500']
X=data[['France_pct_lag1','France_pct_lag2','France_pct_lag3','sigma_sp500_lag1','sigma_sp500_lag2','sigma_sp500_lag3']]
x=sm.add_constant(X)
model=sm.OLS(y,x,missing='drop').fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              ret_sp500   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.8461
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.539
Time:                        16:04:28   Log-Likelihood:                 118.01
No. Observations:                  73   AIC:                            -222.0
Df Residuals:                      66   BIC:                            -206.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0403      0.026  

In [ ]:
y=data['ret_sp500']
X=data[['US_pct_lag1','US_pct_lag2','US_pct_lag3','sigma_sp500_lag1','sigma_sp500_lag2','sigma_sp500_lag3']]
x=sm.add_constant(X)
model=sm.OLS(y,x,missing='drop').fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              ret_sp500   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                    0.7209
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.634
Time:                        16:04:30   Log-Likelihood:                 117.62
No. Observations:                  73   AIC:                            -221.2
Df Residuals:                      66   BIC:                            -205.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0438      0.028  

In [ ]:
y=data['ret_sp500']
X=data[['Singapore_pct_lag1','Singapore_pct_lag2','Singapore_pct_lag3','sigma_sp500_lag1','sigma_sp500_lag2','sigma_sp500_lag3']]
x=sm.add_constant(X)
model=sm.OLS(y,x,missing='drop').fit()
print(model.summary())
#依舊是新加坡相較於其他國家較顯著

                            OLS Regression Results                            
Dep. Variable:              ret_sp500   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.9490
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.467
Time:                        16:04:31   Log-Likelihood:                 118.32
No. Observations:                  73   AIC:                            -222.6
Df Residuals:                      66   BIC:                            -206.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.0262      0

In [ ]:
#& (data['Date'] <= '2022-06-30'

# 建立事件研究 dummy
# 2022 年 3 月事件當月
data['dummy_event_march'] = (data['Date'] >= '2022-03-31').astype(int)


# 移除缺值資料
df_model = data.dropna().copy()

# 定義 y 與 X（加入兩個 dummy）
y = df_model['sigma_sp500']
X_vars = [f'sigma_sp500_lag{i}' for i in range(1, 4)] + \
         [f'France_pct_lag{i}' for i in range(1, 4)] + \
         ['dummy_event_march']

X = df_model[X_vars]
X = sm.add_constant(X)

# 建立 OLS 模型（使用 HAC 修正標準誤）
model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 3})

# 輸出結果
print(model.summary())



                            OLS Regression Results                            
Dep. Variable:            sigma_sp500   R-squared:                       0.893
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     191.0
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           2.31e-40
Time:                        16:05:56   Log-Likelihood:                 289.46
No. Observations:                  72   AIC:                            -562.9
Df Residuals:                      64   BIC:                            -544.7
Df Model:                           7                                         
Covariance Type:                  HAC                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0043      0.00

| 變數                  | 係數        | P 值       | 顯著性             |
| ------------------- | --------- | --------- | --------------- |
| `sigma_sp500_lag1`  | 1.3934    | **0.000** | ✅ \*\*\*        |
| `sigma_sp500_lag2`  | -0.5538   | **0.022** | ✅ \*            |
| `sigma_sp500_lag3`  | 0.0740    | 0.570     | 不顯著               |
| `France_pctlags`    | 全部 > 0.31 | 不顯著       | 不顯著               |
| `dummy_event_march` | -0.0003   | 0.794     | 不顯著               |
| `const`             | 0.0043    | 0.068     | ✳️ 邊緣顯著（10% 水準） |


經濟解釋與意涵
1. sigma_sp500_lag1 與 sigma_sp500_lag2 顯著：
第一期的 正向顯著係數（+1.3934） 表示：如果上一期（t-1）波動性上升，下一期（t+1）也傾向維持高波動，代表市場具有波動性持續性（volatility persistence），這與金融市場中的 ARCH/GARCH 理論相符。

第二期的 負向顯著係數（−0.5538） 暗示：過去第二期（t-2）波動性若偏高，可能會拉回當前波動，顯示市場存在某種均值回歸或修正效果（mean-reverting correction）。

→ 結合兩者，說明 S&P500 的波動性行為並非單純線性累積，而是呈現短期動能 + 中期修正的結構，具有動態調整特性。



###相關係數模型

In [ ]:
#corr_TW_US
window_size = 12

# 計算滾動相關係數
data['corr_TW_US'] = data['ret_sp500'].rolling(window=window_size).corr(data['ret_TW_index'])

# 建立 lag 與 lead 變數
for i in range(1, 4):
    data[f'corr_TW_US_lag{i}'] = data['corr_TW_US'].shift(i)

# 應變數：下一期相關係數
data['corr_TW_US_lead1'] = data['corr_TW_US'].shift(-1)

In [ ]:
y=data['ret_sp500']
X=data[['Australia_pct_lag1','Australia_pct_lag2','Australia_pct_lag3','corr_TW_US_lag1','corr_TW_US_lag2','corr_TW_US_lag3']]
x=sm.add_constant(X)
model=sm.OLS(y,x,missing='drop').fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              ret_sp500   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.017
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.422
Time:                        16:00:44   Log-Likelihood:                 118.53
No. Observations:                  73   AIC:                            -223.1
Df Residuals:                      66   BIC:                            -207.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.0615      0

In [ ]:
y=data['ret_sp500']
X=data[['Brazil_pct_lag1','Brazil_pct_lag2','Brazil_pct_lag3','corr_TW_US_lag1','corr_TW_US_lag2','corr_TW_US_lag3']]
x=sm.add_constant(X)
model=sm.OLS(y,x,missing='drop').fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              ret_sp500   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     1.804
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.112
Time:                        16:00:45   Log-Likelihood:                 120.84
No. Observations:                  73   AIC:                            -227.7
Df Residuals:                      66   BIC:                            -211.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0628      0.034     

In [ ]:
y=data['ret_sp500']
X=data[['France_pct_lag1','France_pct_lag2','France_pct_lag3','corr_TW_US_lag1','corr_TW_US_lag2','corr_TW_US_lag3']]
x=sm.add_constant(X)
model=sm.OLS(y,x,missing='drop').fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              ret_sp500   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.126
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.357
Time:                        16:00:47   Log-Likelihood:                 118.86
No. Observations:                  73   AIC:                            -223.7
Df Residuals:                      66   BIC:                            -207.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0616      0.034     

In [ ]:
y=data['ret_sp500']
X=data[['Singapore_pct_lag1','Singapore_pct_lag2','Singapore_pct_lag3','corr_TW_US_lag1','corr_TW_US_lag2','corr_TW_US_lag3']]
x=sm.add_constant(X)
model=sm.OLS(y,x,missing='drop').fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              ret_sp500   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     1.282
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.278
Time:                        16:00:48   Log-Likelihood:                 119.32
No. Observations:                  73   AIC:                            -224.6
Df Residuals:                      66   BIC:                            -208.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.0472      0

In [ ]:
y=data['ret_sp500']
X=data[['US_pct_lag1','US_pct_lag2','US_pct_lag3','corr_TW_US_lag1','corr_TW_US_lag2','corr_TW_US_lag3']]
x=sm.add_constant(X)
model=sm.OLS(y,x,missing='drop').fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              ret_sp500   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     1.136
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.351
Time:                        16:00:51   Log-Likelihood:                 118.89
No. Observations:                  73   AIC:                            -223.8
Df Residuals:                      66   BIC:                            -207.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0674      0.036     

In [ ]:
# 建立事件研究 dummy
# 2022 年 3 月事件當月
data['dummy_event_march'] = (data['Date'] >= '2022-03-31').astype(int)


# 移除缺值資料
df_model = data.dropna().copy()

# 定義 y 與 X（加入兩個 dummy）
y = df_model['sigma_sp500']
X_vars = [f'corr_TW_US_lag{i}' for i in range(1, 4)] + \
         [f'Singapore_pct_lag{i}' for i in range(1, 4)] + \
         ['dummy_event_march']

X = df_model[X_vars]
X = sm.add_constant(X)

# 建立 OLS 模型（使用 HAC 修正標準誤）
model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 3})

# 輸出結果
print(model.summary())



                            OLS Regression Results                            
Dep. Variable:            sigma_sp500   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     2.492
Date:                Wed, 04 Jun 2025   Prob (F-statistic):             0.0250
Time:                        15:47:32   Log-Likelihood:                 165.81
No. Observations:                  72   AIC:                            -315.6
Df Residuals:                      64   BIC:                            -297.4
Df Model:                           7                                         
Covariance Type:                  HAC                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0115      0

###相關係數模型結論

| 變數名稱                     | 係數      | p 值   | 顯著性         | 解釋                       |
| ------------------------ | ------- | ----- | ----------- | ------------------------ |
| **corr\_TW\_US\_lag1**   | 0.1174  | 0.001 |  **1% 顯著** | 台美相關性上升 → **隔期美股波動性上升**  |
| corr\_TW\_US\_lag2       | -0.0346 | 0.389 | 不顯著          | 無明顯效果                    |
| corr\_TW\_US\_lag3       | -0.0394 | 0.315 | 不顯著          | 無明顯效果                    |
| **Singapore\_pct\_lag1** | 0.0493  | 0.004 |  **1% 顯著** | 新加坡政策變動率上升 → **波動性同步放大** |
| **Singapore\_pct\_lag2** | 0.0354  | 0.015 |  **5% 顯著** | 滯後第二期仍有推動波動性效果           |
| Singapore\_pct\_lag3     | -0.0091 | 0.379 |   不顯著        | 無顯著性                     |
| dummy\_event\_march      | 0.0058  | 0.598 |  不顯著         | 事件虛擬變數無效                 |


回歸結果顯示，台美報酬率相關性對美股波動具有顯著預測力，尤其第一期滯後效果達到 1% 顯著水準（p = 0.001）。此外，新加坡政策風險變動對 S&P500 波動的影響亦顯著，lag1 與 lag2 均在 5% 顯著水準內。整體而言，台美市場連動性與新加坡風險變動皆為驅動美股波動的關鍵因素。